In [34]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime

stocks = ['AMD', 'NVDA', 'META', 'TSLA']
start = datetime(2018, 1, 1)
end = datetime(2024, 4, 30)

stock_prices = yf.download(stocks, start, end, auto_adjust=False)
stock_prices = stock_prices.resample('ME').last()
stock_prices.index = stock_prices.index.tz_localize(None)
stock_prices = stock_prices.filter(like='Adj Close')
stock_prices.columns = stock_prices.columns.get_level_values(1)

pairs = [('AMD', 'NVDA'), ('AMD', 'META'), ('NVDA', 'TSLA')]
spread_list = []

for pair in pairs:
    ticker_1, ticker_2 = pair
    spread = np.log(stock_prices[ticker_1]) - np.log(stock_prices[ticker_2])
    
    spread_mean = spread.expanding().mean()
    spread_std = spread.expanding().std()
    z_score = (spread - spread_mean) / spread_std

    position = np.where(z_score > 0, 'Sell', 'Buy')
    
    pair_df = pd.DataFrame({
        'Date': spread.index,
        'Ticker Pair': f'{ticker_1}-{ticker_2}',
        'Traditional Spread': spread.values,
        'Traditional Z-score': z_score.values,
        'Traditional Position': position
    })

    spread_list.append(pair_df)

full_traditional_df = pd.concat(spread_list)
full_traditional_df.to_csv("outputs/traditional_dummy2.csv", index=False)

[*********************100%***********************]  4 of 4 completed


In [56]:
full_traditional_df.tail()

,Date,Ticker Pair,Traditional Spread,Traditional Z-score,Traditional Position
71,2023-12-31,NVDA-TSLA,-1.613321,0.665554,Sell
72,2024-01-31,NVDA-TSLA,-1.113557,1.485575,Sell
73,2024-02-29,NVDA-TSLA,-0.937184,1.734942,Sell
74,2024-03-31,NVDA-TSLA,-0.665861,2.106395,Sell
75,2024-04-30,NVDA-TSLA,-0.793873,1.841791,Sell


In [58]:
df = pd.read_csv("outputs/traditional_dummy2.csv", parse_dates=["Date"])
returns_raw = pd.read_csv("outputs/returns.csv", parse_dates=["Date"])

returns_wide = returns_raw.pivot(index="Date", columns="Ticker", values="Return")

pairs = [('AMD', 'NVDA'), ('AMD', 'META'), ('NVDA', 'TSLA')]
returns_list = []

for t1, t2 in pairs:
    temp = pd.DataFrame()
    temp["Date"] = returns_wide.index
    temp["Return"] = returns_wide[t1] - returns_wide[t2]  # return difference
    temp["Ticker Pair"] = f"{t1}-{t2}"
    returns_list.append(temp)

returns_ready = pd.concat(returns_list)

merged_df = df.merge(returns_ready, on=["Date", "Ticker Pair"], how="left")

merged_df["Position"] = np.where(
    merged_df["Traditional Position"] == "Buy", 1,
    np.where(merged_df["Traditional Position"] == "Sell", -1, 0)
)

# Shift position to simulate using previous signal
merged_df["Strategy Return"] = merged_df["Position"].shift(1) * merged_df["Return"]

# Add transaction cost
merged_df["Strategy Net Return"] = (
    merged_df["Strategy Return"] - 0.001 * np.abs(merged_df["Position"].diff().fillna(0))
)

merged_df.to_csv("outputs/traditional_results.csv", index=False)